In [9]:
import numpy as np
import sklearn.gaussian_process.kernels as kernels
import kolesky
import scipy
import scipy.sparse as sparse
from copy import deepcopy

from kolesky.ordering import p_reverse_maximin
from kolesky.ordering import sparsity_pattern
from kolesky.nugget import ichol
from kolesky.nugget import parallel_ichol

In [2]:
def logdet_chol(A):
    return 2 * np.sum(np.log(A.diagonal()))

def kl_div(A, B):
    n = A.shape[0]
    return 0.5 * (np.trace(np.linalg.solve(B, A)) - n + np.linalg.slogdet(B)[1] - np.linalg.slogdet(A)[1])

def sparse_kl_div(A, L):
    n = A.shape[0]
    return 0.5 * (-logdet_chol(L) - np.linalg.slogdet(A)[1])

In [32]:
np.random.seed(0)
n = 75
points = np.zeros((n * n, 2))
for i in range(n):
    for j in range(n):
        perturbation = np.random.uniform(-0.2, 0.2, 2)
        points[i * n + j] = np.array([i - n/2, j - n/2]) + perturbation
noise = np.eye(len(points)) * 0.3

In [13]:
def __supernodes(sparsity, lengths, lamb):
    groups = []
    candidates = set(range(len(lengths)))
    agg_sparsity = {}
    i = 0
    while len(candidates) > 0:
        while i not in candidates:
            i += 1
        group = sorted(j for j in sparsity[i] if lengths[j] <= lamb * lengths[i] and j in candidates)
        groups.append(group)
        candidates -= set(group)
        s = sorted({k for j in group for k in sparsity[j]})
        agg_sparsity[group[0]] = s
        positions = {k: j for j, k in enumerate(s)}
        for j in group[1:]:
            agg_sparsity[j] = np.empty(len(s) - positions[j], dtype=int)
    return groups, agg_sparsity

def __cols(theta):
    return np.flip(np.linalg.cholesky(np.flip(theta))).T

def __aggregate_chol(points, kernel, sparsity, groups):
    n = len(points)
    ptr = np.cumsum([0] + [len(sparsity[i]) for i in range(n)])
    data, indices = np.zeros(ptr[-1]), np.zeros(ptr[-1])
    for group in groups:
        s = sorted(sparsity[group[0]])
        positions = {i: k for k, i in enumerate(s)}
        L_group = __cols(kernel(points[s]))
        for i in group:
            k = positions[i]
            e_k = np.zeros(len(s))
            e_k[k] = 1
            col = scipy.linalg.solve_triangular(L_group, e_k, lower=True, check_finite=False)
            data[ptr[i] : ptr[i + 1]] = col[k:]
            indices[ptr[i] : ptr[i + 1]] = s[k:]
    return sparse.csc_matrix((data, indices, ptr), shape=(n, n))

def innerprod(iter1, u1, iter2, u2, indices, data):
    prod = 0
    while iter1 <= u1 and iter2 <= u2:
        while iter1 <= u1 and iter2 <= u2 and indices[iter1] == indices[iter2]:
            prod += data[iter1] * data[iter2]
            iter1 += 1
            iter2 += 1
        if indices[iter1] < indices[iter2]:
            iter1 += 1
        else:
            iter2 += 1
    return prod

def py_ichol(A):
    indptr = A.indptr #ind
    indices = A.indices #jnd
    data = A.data
    for i in range(len(indptr) - 1):
        for j in range(indptr[i], indptr[i + 1]):
            iter_i = indptr[i]
            iter_j = indptr[indices[j]]
            data[j] -= innerprod(iter_i, indptr[i + 1] - 2, iter_j, indptr[indices[j] + 1] - 2, indices, data)
            if data[indptr[indices[j] + 1] - 1] > 0:
                if indices[j] < i:
                    data[j] /= data[indptr[indices[j] + 1] - 1]
                    if np.isnan(data[j]) or np.isinf(data[j]):
                        print("nan or inf")
                else:
                    if j != indptr[i + 1] - 1:
                        print("not diagonal")
                    data[j] = np.sqrt(data[j])
            else:
                data[j] = 0

In [29]:
def py_noise_cholesky(points, kernel, rho, lamb, noise, initial = None, p = 1):
    n = len(points)
    indices, lengths = p_reverse_maximin(points, initial, p)
    ordered_points = points[indices]
    sparsity = sparsity_pattern(ordered_points, lengths, rho)
    groups, agg_sparsity = __supernodes(sparsity, lengths, lamb)
    L = __aggregate_chol(ordered_points, kernel, agg_sparsity, groups)
    A = sparse.triu(L @ L.T, format='csc')
    A += sparse.csc_matrix(np.linalg.inv(noise))
    # U = deepcopy(A)
    # parallel_ichol(A.indptr, A.indices, A.data, U.data, sweeps=5)
    # ichol(U.indptr, U.indices, U.data)
    py_ichol(A)
    return L, A, indices

def noise_cholesky(points, kernel, rho, lamb, noise, initial = None, p = 1):
    n = len(points)
    indices, lengths = p_reverse_maximin(points, initial, p)
    ordered_points = points[indices]
    sparsity = sparsity_pattern(ordered_points, lengths, rho)
    groups, agg_sparsity = __supernodes(sparsity, lengths, lamb)
    L = __aggregate_chol(ordered_points, kernel, agg_sparsity, groups)
    A = sparse.triu(L @ L.T, format='csc')
    A += sparse.csc_matrix(np.linalg.inv(noise))
    # U = deepcopy(A)
    # parallel_ichol(A.indptr, A.indices, A.data, U.data, sweeps=5)
    ichol(A.indptr, A.indices, A.data)
    # py_ichol(A)
    return L, A, indices

def parallel_noise_cholesky(points, kernel, rho, lamb, noise, initial = None, p = 1, sweeps=5):
    n = len(points)
    indices, lengths = p_reverse_maximin(points, initial, p)
    ordered_points = points[indices]
    sparsity = sparsity_pattern(ordered_points, lengths, rho)
    groups, agg_sparsity = __supernodes(sparsity, lengths, lamb)
    L = __aggregate_chol(ordered_points, kernel, agg_sparsity, groups)
    A = sparse.triu(L @ L.T, format='csc')
    A += sparse.csc_matrix(np.linalg.inv(noise))
    U = deepcopy(A)
    parallel_ichol(A.indptr, A.indices, A.data, U.data, sweeps=sweeps)
    # ichol(A.indptr, A.indices, A.data)
    # py_ichol(A)
    return L, U, indices

In [15]:
kernel = kernels.Matern(length_scale=1.0, nu=0.5)
rho = 4.0
lamb = 1.5

In [20]:
L, U_tilde, ordering = py_noise_cholesky(points, kernel, rho, lamb, noise)
L = L.toarray()
ordered_points = points[ordering]
true = kernel(ordered_points) + noise
approx = np.linalg.inv(L @ L.T) @ U_tilde.T @ U_tilde @ noise
kl = kl_div(true, approx)
print("KL divergence:", kl)
L, U_tilde, ordering = noise_cholesky(points, kernel, rho, lamb, noise)
L = L.toarray()
approx = np.linalg.inv(L @ L.T) @ U_tilde.T @ U_tilde @ noise
kl = kl_div(true, approx)
print("KL divergence:", kl)
L, U_tilde, ordering = parallel_noise_cholesky(points, kernel, rho, lamb, noise)
L = L.toarray()
approx = np.linalg.inv(L @ L.T) @ U_tilde.T @ U_tilde @ noise
kl = kl_div(true, approx)
print("KL divergence:", kl)

/tmp/ipykernel_7739/1092571473.py:3: DeprecationWarning: `np.long` is a deprecated alias for `np.compat.long`. To silence this warning, use `np.compat.long` by itself. In the likely event your code does not need to work on Python 2 you can use the builtin `int` for which `np.compat.long` is itself an alias. Doing this will not modify any behaviour and is safe. When replacing `np.long`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  indices, lengths = p_reverse_maximin(points, initial, p)
/tmp/ipykernel_7739/1092571473.py:18: DeprecationWarning: `np.long` is a deprecated alias for `np.compat.long`. To silence this warning, use `np.compat.long` by itself. In the likely event your code does not need to work on Python 2 you can use the builtin `

KL divergence: 6.691487264731677e-06
KL divergence: 6.691487264731677e-06


/tmp/ipykernel_7739/1092571473.py:33: DeprecationWarning: `np.long` is a deprecated alias for `np.compat.long`. To silence this warning, use `np.compat.long` by itself. In the likely event your code does not need to work on Python 2 you can use the builtin `int` for which `np.compat.long` is itself an alias. Doing this will not modify any behaviour and is safe. When replacing `np.long`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  indices, lengths = p_reverse_maximin(points, initial, p)


KL divergence: 6.69173566958392e-06


In [21]:
%timeit py_noise_cholesky(points, kernel, rho, lamb, noise)

/tmp/ipykernel_7739/1092571473.py:3: DeprecationWarning: `np.long` is a deprecated alias for `np.compat.long`. To silence this warning, use `np.compat.long` by itself. In the likely event your code does not need to work on Python 2 you can use the builtin `int` for which `np.compat.long` is itself an alias. Doing this will not modify any behaviour and is safe. When replacing `np.long`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  indices, lengths = p_reverse_maximin(points, initial, p)


187 ms ± 7.97 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [33]:
%timeit noise_cholesky(points, kernel, rho, lamb, noise)

/tmp/ipykernel_7739/8132028.py:18: DeprecationWarning: `np.long` is a deprecated alias for `np.compat.long`. To silence this warning, use `np.compat.long` by itself. In the likely event your code does not need to work on Python 2 you can use the builtin `int` for which `np.compat.long` is itself an alias. Doing this will not modify any behaviour and is safe. When replacing `np.long`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  indices, lengths = p_reverse_maximin(points, initial, p)


5.91 s ± 338 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [34]:
%timeit parallel_noise_cholesky(points, kernel, rho, lamb, noise, sweeps=5)

/tmp/ipykernel_7739/8132028.py:33: DeprecationWarning: `np.long` is a deprecated alias for `np.compat.long`. To silence this warning, use `np.compat.long` by itself. In the likely event your code does not need to work on Python 2 you can use the builtin `int` for which `np.compat.long` is itself an alias. Doing this will not modify any behaviour and is safe. When replacing `np.long`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  indices, lengths = p_reverse_maximin(points, initial, p)


6.57 s ± 363 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
